In [2]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

c:\Users\admin\Desktop\p_chatbot\chatbot-dl\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 모델 가져오기

In [3]:
# bos_token 등은 모델마다 달라서 아래와 같이 설정
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2',
                                            bos_token='<s>',
                                            eos_token='</s>',
                                            pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT

In [4]:
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  125164032 
 er)                                                             
                                                                 
Total params: 125164032 (477.46 MB)
Trainable params: 125164032 (477.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# tokenizer
tokenizer.vocab_size

51200

In [6]:
import pandas as pd

data1 = pd.read_csv("data/chatbot_data.csv")
data1.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [7]:
data2 = pd.read_csv("data/all_data.csv")
data2.head()

,req,res
0,너 좋아하는 차 종류 있어?,무슨 차? 자동차? 마시는 차?
1,ㅋㅋ 마시는 차 말한 거야!,"아하 나 둥글레, 옥수수, 보리차 좋아해"
2,완전 곡물류 좋아하네 ㅋㅋ,야쓰 끓이기 귀찮아서 냉침해 먹어
3,그럼 오래 걸리지 않아?,끓이는 것보다는 훨씬 오래 걸리지 ㅠ
4,근데 냉침 하는 것도 귀찮겠다 ㅜㅠ,응! 그래서 매일은 안 먹고 가끔 마셔


In [12]:
chat_data = list(zip(data1['Q'].to_list(), data1['A'].to_list()))
print(chat_data[:1])
chat_data2 = list(zip(data2['req'].to_list(), data2['res'].to_list()))
print(chat_data2[:1])
chat_data.extend(chat_data2)
len(chat_data)

[('12시 땡!', '하루가 또 가네요.')]
[('너 좋아하는 차 종류 있어?', '무슨 차? 자동차? 마시는 차?')]


100797

In [15]:
def get_chat_data():
    bos_token = tokenizer.bos_token
    eos_token = tokenizer.eos_token
    for question, answer in chat_data:
        sentence = f"{bos_token}<usr>{question}<sys>{answer}{eos_token}"
        # print(sentence)
        yield tokenizer.encode(sentence)

In [16]:
gen = get_chat_data()
next(gen)

[0, 2, 9349, 7888, 739, 7318, 376, 4, 12557, 6824, 9108, 9028, 7098, 25856, 1]

In [18]:
batch_size = 32
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

## 학습 시키기

In [23]:
# 옵티마이저 설정
import keras
adam = keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

전이 학습은 learning rate를 아주 작게 설정해, 가중치를 미세하게 조금씩 조금씩 조정한다.

- 사전 학습된 가중치 보존이 중요하기 때문 (기존의 가중치가 유용할 것이므로)

In [ ]:
# 학습

steps = len(chat_data) // batch_size  # 1 epoch마다 학습할 batch 사이즈
epochs = 20  # 너무 오래 걸려서 일단 1회만

for epoch in range(epochs):
    train_loss = 0

    for batch in dataset:
        with tf.GradientTape() as tape:
            result = model(batch, labels=batch)
            batch_loss = tf.reduce_mean(result[0])
            
            grads = tape.gradient(batch_loss, model.trainable_variables)
            adam.apply_gradients(zip(grads, model.trainable_variables))

        train_loss += batch_loss
    
    train_loss /= steps

    # 조기종료는 하지 않는다.

    print(f"Epoch {epoch+1}/{epochs} - loss: {train_loss:.4f}")

KeyboardInterrupt: 

In [ ]:
model.save_weights("chatbot")